#Imports

In [ ]:
import pandas as pd
import numpy as np

seed = 93646

# Carga de datos

In [ ]:
shap_xg_boost = pd.read_csv("https://raw.githubusercontent.com/FranArenas/Fran_Arenas_XAI_credit_card/main/results/shap_xg_boost.csv")
shap_dt = pd.read_csv("https://raw.githubusercontent.com/FranArenas/Fran_Arenas_XAI_credit_card/main/results/shap_dt.csv")
shap_mlp = pd.read_csv("https://raw.githubusercontent.com/FranArenas/Fran_Arenas_XAI_credit_card/main/results/shap_mlp.csv")
shap_knn = pd.read_csv("https://raw.githubusercontent.com/FranArenas/Fran_Arenas_XAI_credit_card/main/results/shap_knn.csv")
shap_rf = pd.read_csv("https://raw.githubusercontent.com/FranArenas/Fran_Arenas_XAI_credit_card/main/results/shap_rf.csv")
shap_mean_values = pd.read_csv("https://raw.githubusercontent.com/FranArenas/Fran_Arenas_XAI_credit_card/main/results/shap_mean_values.csv")

# Análisis de datos

In [ ]:
shap_mean_values.pop("Unnamed: 0");

In [ ]:
shap_mean_values.head()

,CODE_GENDER_0,CODE_GENDER_1,FLAG_OWN_CAR_0,FLAG_OWN_CAR_1,FLAG_OWN_REALTY_0,FLAG_OWN_REALTY_1,CNT_CHILDREN_0,CNT_CHILDREN_1,CNT_CHILDREN_2,CNT_CHILDREN_3,CNT_CHILDREN_4,CNT_CHILDREN_5,CNT_CHILDREN_7,CNT_CHILDREN_14,CNT_CHILDREN_19,NAME_EDUCATION_TYPE_0,NAME_EDUCATION_TYPE_1,NAME_EDUCATION_TYPE_2,NAME_EDUCATION_TYPE_3,NAME_EDUCATION_TYPE_4,NAME_FAMILY_STATUS_0,NAME_FAMILY_STATUS_1,NAME_FAMILY_STATUS_2,NAME_FAMILY_STATUS_3,NAME_FAMILY_STATUS_4,NAME_HOUSING_TYPE_0,NAME_HOUSING_TYPE_1,NAME_HOUSING_TYPE_2,NAME_HOUSING_TYPE_3,NAME_HOUSING_TYPE_4,NAME_HOUSING_TYPE_5,OCCUPATION_TYPE_0,OCCUPATION_TYPE_1,OCCUPATION_TYPE_2,OCCUPATION_TYPE_3,OCCUPATION_TYPE_4,OCCUPATION_TYPE_5,OCCUPATION_TYPE_6,OCCUPATION_TYPE_7,OCCUPATION_TYPE_8,...,YEARS_EMPLOYED_4,YEARS_EMPLOYED_5,YEARS_EMPLOYED_6,YEARS_EMPLOYED_7,YEARS_EMPLOYED_8,YEARS_EMPLOYED_9,YEARS_EMPLOYED_10,YEARS_EMPLOYED_11,YEARS_EMPLOYED_12,YEARS_EMPLOYED_13,YEARS_EMPLOYED_14,YEARS_EMPLOYED_15,YEARS_EMPLOYED_16,YEARS_EMPLOYED_17,YEARS_EMPLOYED_18,YEARS_EMPLOYED_19,YEARS_EMPLOYED_20,YEARS_EMPLOYED_21,YEARS_EMPLOYED_22,YEARS_EMPLOYED_23,YEARS_EMPLOYED_24,YEARS_EMPLOYED_25,YEARS_EMPLOYED_26,YEARS_EMPLOYED_27,YEARS_EMPLOYED_28,YEARS_EMPLOYED_29,YEARS_EMPLOYED_30,YEARS_EMPLOYED_31,YEARS_EMPLOYED_32,YEARS_EMPLOYED_33,YEARS_EMPLOYED_34,YEARS_EMPLOYED_35,YEARS_EMPLOYED_36,YEARS_EMPLOYED_37,YEARS_EMPLOYED_38,YEARS_EMPLOYED_39,YEARS_EMPLOYED_40,YEARS_EMPLOYED_42,YEARS_EMPLOYED_43,Model
0,-0.039346,-0.016938,-0.066447,-0.008662,-0.100953,0.092033,0.028002,-0.314470,-0.097814,-0.310104,-0.528729,-0.197377,-0.483968,-0.046357,0.196301,-0.262607,-0.228570,-0.231800,-0.341629,0.240200,-0.057373,-0.059417,-0.047386,-0.051521,-0.068391,-0.045368,-0.039195,-0.021567,-0.074829,-0.158019,-0.087789,-0.053093,-0.060546,-0.076058,-0.048772,-0.055733,-0.049622,-0.049513,-0.039157,-0.064886,...,-0.041099,-0.044723,-0.030904,-0.024279,-0.031002,-0.030930,-0.066783,-0.010591,-0.034125,-0.034451,-0.037938,-0.048017,-0.033108,-0.025131,-0.053161,-0.037299,-0.021751,-0.021131,-0.035796,-0.038444,-0.019777,-0.033025,-0.031216,-0.043856,-0.011645,0.025741,-0.106636,-0.126085,-0.219060,0.007757,-0.023392,-0.004820,-0.026872,-0.023445,-0.033350,0.034645,-0.070482,0.044409,-0.055570,xgboost
1,-0.002027,0.003805,0.002193,-0.000556,0.006908,0.002538,0.009944,0.000677,0.008939,0.004404,0.002333,0.280662,0.069609,0.046856,-0.003183,0.027366,0.020760,0.024267,0.032889,-0.070399,0.003655,0.005743,0.004821,0.004246,0.001574,0.006387,0.004386,0.000561,0.013998,0.008529,0.016813,0.001606,0.000539,-0.003481,0.003065,-0.000435,0.002147,-0.001493,0.002150,0.001038,...,0.004192,-0.001550,0.001295,0.000111,0.000281,0.000402,-0.003129,0.002765,0.002154,-0.000214,0.000742,-0.000672,0.000196,0.000458,-0.001627,-0.001403,-0.005974,-0.000201,0.000449,-0.000952,0.007380,0.001210,-0.000617,-0.001870,0.001566,0.011620,-0.003890,-0.001116,-0.005521,0.002874,0.000518,-0.000340,-0.000855,0.002409,0.001675,0.002014,-0.000736,0.000143,-0.000062,DT
2,-0.020101,-0.053200,-0.034498,-0.006513,-0.023771,0.001792,-0.031073,0.033802,0.004404,0.033010,0.015916,-0.173795,-0.056429,-0.139081,0.025637,-0.025092,-0.008950,-0.007765,-0.020330,0.010637,-0.022035,-0.025166,-0.028035,-0.025032,-0.023761,0.016319,0.019289,0.017783,0.003765,-0.004080,0.001534,0.015865,0.002517,0.003499,0.008781,0.006649,0.002291,0.005334,0.006605,0.000808,...,-0.012382,-0.019407,-0.010661,-0.004968,-0.005727,-0.006900,-0.027177,0.008749,-0.015712,-0.012360,-0.011821,-0.015760,-0.007479,-0.015546,-0.014612,-0.009596,-0.016433,-0.010823,-0.004888,-0.012070,0.000577,-0.012480,-0.008243,-0.021649,-0.000448,0.001839,-0.035945,-0.035440,-0.056925,0.014134,0.000560,0.003536,-0.009142,0.005415,-0.001114,0.003482,-0.072681,0.014095,0.003647,MLP
3,-0.004708,-0.006154,-0.005516,-0.002986,-0.009664,-0.002235,-0.009588,-0.051319,-0.002577,-0.021336,-0.104625,0.011146,-0.075657,-0.054248,-0.084048,-0.124671,-0.128109,-0.093727,-0.126105,0.249226,-0.006220,-0.004794,-0.004538,-0.002917,-0.004148,-0.00

#Escalamos los valores entre 100 y -100

In [ ]:
from sklearn.preprocessing import normalize

shap_mean_norm = shap_mean_values.copy()

shap_mean_norm.pop("Model")

cols_names = shap_mean_norm.columns

shap_mean_norm = normalize(shap_mean_norm)*100

shap_mean_norm = pd.DataFrame(shap_mean_norm, columns = cols_names)

## Métricas

In [ ]:
from sklearn.metrics import r2_score,mean_squared_error, mean_absolute_error

for i in range(shap_mean_norm.shape[0]):
  for j in range(shap_mean_norm.shape[0]):
    if i != j:
      r2 = r2_score(shap_mean_norm.iloc[i][:].to_numpy(),shap_mean_norm.iloc[j][:].to_numpy())
      mse = mean_squared_error(shap_mean_norm.iloc[i][:].to_numpy(),shap_mean_norm.iloc[j][:].to_numpy())
      mae = mean_absolute_error(shap_mean_norm.iloc[i][:].to_numpy(),shap_mean_norm.iloc[j][:].to_numpy())

      name1 = shap_mean_values.iloc[i][-1:].to_numpy()[0]
      name2 = shap_mean_values.iloc[j][-1:].to_numpy()[0]

      print("r2 Score for {} and {} : {}".format(name1,name2,r2))
      print("Mean squared error for {} and {} : {}".format(name1,name2,mse))
      print("Mean absolute error for {} and {} : {}".format(name1,name2,mae))
      print("\n")

r2 Score for xgboost and DT : -2.725319324844746
Mean squared error for xgboost and DT : 198.08149938453263
Mean absolute error for xgboost and DT : 6.782446089571855


r2 Score for xgboost and MLP : -0.8146972934848373
Mean squared error for xgboost and MLP : 96.49050980012576
Mean absolute error for xgboost and MLP : 5.3778301126089465


r2 Score for xgboost and KNN : 0.05387685053764124
Mean squared error for xgboost and KNN : 50.30696047934915
Mean absolute error for xgboost and KNN : 3.7994892812769656


r2 Score for DT and xgboost : -1.8438338302959645
Mean squared error for DT and xgboost : 198.08149938453263
Mean absolute error for DT and xgboost : 6.782446089571855


r2 Score for DT and MLP : -2.3594482468343965
Mean squared error for DT and MLP : 233.99557975173337
Mean absolute error for DT and MLP : 6.28425891312816


r2 Score for DT and KNN : -1.633271569049478
Mean squared error for DT and KNN : 183.41521052574902
Mean absolute error for DT and KNN : 4.973010457793383


r

# Valores de alto impacto

In [ ]:
vals_max = []
for i in shap_mean_norm.max():
  vals_max.append(i)

vals_min = []
for i in shap_mean_norm.min():
  vals_min.append(i)

In [ ]:
shap_mean_norm["YEARS_EMPLOYED_34"].max() > 0

True

In [ ]:
high_cols = []
for col in shap_mean_norm.columns:
  c_col = shap_mean_norm[col]
  if c_col.max() >= 15  or c_col.min() <= -15:
    high_cols.append(c_col.name)

In [ ]:
shap_mean_norm[high_cols]

,CODE_GENDER_1,CNT_CHILDREN_1,CNT_CHILDREN_3,CNT_CHILDREN_4,CNT_CHILDREN_5,CNT_CHILDREN_7,CNT_CHILDREN_14,CNT_CHILDREN_19,NAME_EDUCATION_TYPE_0,NAME_EDUCATION_TYPE_1,NAME_EDUCATION_TYPE_2,NAME_EDUCATION_TYPE_3,NAME_EDUCATION_TYPE_4,YEARS_EMPLOYED_32,YEARS_EMPLOYED_40
0,-1.391938,-25.843209,-25.484433,-43.451090,-16.220473,-39.772630,-3.809647,16.132100,-21.581122,-18.783931,-19.049384,-28.075121,19.739698,-18.002374,-5.792211
1,1.226373,0.218315,1.419270,0.752015,90.454196,22.434073,15.101327,-1.025974,8.819860,6.690738,7.820875,10.599901,-22.688771,-1.779503,-0.237157
2,-17.435391,11.078085,10.818327,5.216209,-56.957976,-18.493496,-45.581261,8.402075,-8.223571,-2.933209,-2.544716,-6.662744,3.486068,-18.656020,-23.819770
3,-1.561304,-13.020069,-5.413128,-26.544402,2.827844,-19.195020,-13.763332,-21.323886,-31.630285,-32.502516,-23.779392,-31.994049,63.231086,-0.896196,-0.836099


# To do

- Observar si las variables falladas y predecidas de los modelos son las mismas en su mayoría. De esta manera se podrá saber si las diferencias en los shapley values dependen del tipo de modelo o de los datos

# Prueba con Titanic dataset

In [ ]:
df= pd.read_csv("https://gist.githubusercontent.com/michhar/2dfd2de0d4f8727f873422c5d959fff5/raw/fa71405126017e6a37bea592440b4bee94bf7b9e/titanic.csv")

df.pop('Name')

df = pd.get_dummies(df, dummy_na= True).dropna()

df.head()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Sex_nan,Ticket_110152,Ticket_110413,Ticket_110465,Ticket_110564,Ticket_110813,Ticket_111240,Ticket_111320,Ticket_111361,Ticket_111369,Ticket_111426,Ticket_111427,Ticket_111428,Ticket_112050,Ticket_112052,Ticket_112053,Ticket_112058,Ticket_112059,Ticket_112277,Ticket_112379,Ticket_113028,Ticket_113043,Ticket_113050,Ticket_113051,Ticket_113055,Ticket_113056,Ticket_113059,Ticket_113501,Ticket_113503,Ticket_113505,Ticket_113509,...,Cabin_D6,Cabin_D7,Cabin_D9,Cabin_E10,Cabin_E101,Cabin_E12,Cabin_E121,Cabin_E17,Cabin_E24,Cabin_E25,Cabin_E31,Cabin_E33,Cabin_E34,Cabin_E36,Cabin_E38,Cabin_E40,Cabin_E44,Cabin_E46,Cabin_E49,Cabin_E50,Cabin_E58,Cabin_E63,Cabin_E67,Cabin_E68,Cabin_E77,Cabin_E8,Cabin_F E69,Cabin_F G63,Cabin_F G73,Cabin_F2,Cabin_F33,Cabin_F38,Cabin_F4,Cabin_G6,Cabin_T,Cabin_nan,Embarked_C,Embarked_Q,Embarked_S,Embarked_nan
0,1,0,3,22.0,1,0,7.2500,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
1,2,1,1,38.0,1,0,71.2833,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,3,1,3,26.0,0,0,7.9250,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
3,4,1,1,35.0,1,0,53.1000,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,5,0,3,35.0,0,0,8.0500,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0


In [ ]:
X = df.drop('Survived', axis= 1)
y = df['Survived']

# Entrenamos nuestros modelos

In [ ]:
from sklearn.model_selection import train_test_split
!pip install shap
import shap

     |████████████████████████████████| 358kB 8.3MB/s 
  Created wheel for shap: filename=shap-0.39.0-cp37-cp37m-linux_x86_64.whl size=491623 sha256=f8b38798f7122d83f9570e0e1d0a26ce6e01e6e7ad2e0f38accb0a6c9162b584
  Stored in directory: /root/.cache/pip/wheels/15/27/f5/a8ab9da52fd159aae6477b5ede6eaaec69fd130fa0fa59f283
Successfully built shap


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = seed)

In [ ]:
def variable_impact(shap_values,X,columns):
  impact = []
  for col,feature in enumerate(columns):
    for unique in X[feature].unique():
      index = X[X[feature] == unique][feature].index
      mean = 0
      for i in index:
        mean += shap_values[i][col]
      mean = mean/len(index)
      impact.append(mean)
  return impact

# XGBoost

In [ ]:
from sklearn.metrics import confusion_matrix
from xgboost import XGBClassifier

In [ ]:
xgb_class = XGBClassifier()

xgb_class.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
confusion_matrix(y_test, xgb_class.predict(X_test))

array([[95, 12],
       [21, 51]])

In [ ]:
#Dependiendo de la cantadidad de árboles usado en nuestro modelo puede tardar una cantidad de tiempo elevada
explainer_xgboost = shap.TreeExplainer(xgb_class)
shap_values_xgboost_all = explainer_xgboost.shap_values(X)


shap_values_xgboost = np.zeros(len(shap_values_xgboost_all[0]))
for i,_ in enumerate(xgb_class.predict(X)):
    shap_values_xgboost += shap_values_xgboost_all[i]

mean_xgb_shap = shap_values_xgboost/len(xgb_class.predict(X))
    

## Random forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf_model = RandomForestClassifier()

rf_model.fit(X_train,y_train)

confusion_matrix(y_test, rf_model.predict(X_test))

array([[97, 10],
       [23, 49]])

In [ ]:
explainer_rf = shap.Explainer(rf_model)
shap_values_rf_all = explainer_rf.shap_values(X, check_additivity=False)
preds = rf_model.predict(X)
shap_values_rf_t = []
for i,_ in enumerate(shap_values_rf_all[0]):
  shap_values_rf_t.append(shap_values_rf_all[preds[i]][i])

shap_values_rf = np.zeros(len(shap_values_rf_t[0]))
for i,_ in enumerate(rf_model.predict(X)):
    shap_values_rf += shap_values_rf_t[i]

mean_shap_rf = shap_values_rf/len(preds)

## Analisis Shap values

In [ ]:
final_df = pd.DataFrame(data= [mean_xgb_shap, mean_shap_rf],columns = X.columns)

In [ ]:
from sklearn.preprocessing import normalize

final_df_norm = final_df.copy()

final_df_norm = normalize(final_df_norm)*100

final_df_norm = pd.DataFrame(final_df_norm, columns = final_df.columns)

In [ ]:
final_df_norm

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Sex_nan,Ticket_110152,Ticket_110413,Ticket_110465,Ticket_110564,Ticket_110813,Ticket_111240,Ticket_111320,Ticket_111361,Ticket_111369,Ticket_111426,Ticket_111427,Ticket_111428,Ticket_112050,Ticket_112052,Ticket_112053,Ticket_112058,Ticket_112059,Ticket_112277,Ticket_112379,Ticket_113028,Ticket_113043,Ticket_113050,Ticket_113051,Ticket_113055,Ticket_113056,Ticket_113059,Ticket_113501,Ticket_113503,Ticket_113505,Ticket_113509,Ticket_113510,...,Cabin_D6,Cabin_D7,Cabin_D9,Cabin_E10,Cabin_E101,Cabin_E12,Cabin_E121,Cabin_E17,Cabin_E24,Cabin_E25,Cabin_E31,Cabin_E33,Cabin_E34,Cabin_E36,Cabin_E38,Cabin_E40,Cabin_E44,Cabin_E46,Cabin_E49,Cabin_E50,Cabin_E58,Cabin_E63,Cabin_E67,Cabin_E68,Cabin_E77,Cabin_E8,Cabin_F E69,Cabin_F G63,Cabin_F G73,Cabin_F2,Cabin_F33,Cabin_F38,Cabin_F4,Cabin_G6,Cabin_T,Cabin_nan,Embarked_C,Embarked_Q,Embarked_S,Embarked_nan
0,-1.261417,93.323285,-7.930687,12.385224,-0.104845,-25.124918,-13.974091,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.00000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,-13.659429,-7.664288,1.007349,0.000000,0.000000
1,-15.755332,26.270076,9.644891,6.571015,0.493425,8.791347,45.787410,68.100563,0.0,0.234465,0.066651,0.012661,0.0,0.040619,-0.037157,0.024503,0.142251,0.160519,-0.062806,0.0,0.0,0.005804,0.0,0.02466,0.0,-0.024174,0.217858,0.0,0.0,0.005066,0.021913,0.058493,0.282678,0.0,-0.039005,0.011479,0.086423,0.048669,0.019546,0.0,...,0.016724,0.032471,0.0,-2.449288,0.109478,0.0,0.308488,0.260999,0.195003,0.269172,0.009669,0.054935,0.0,0.0,0.00954,0.070363,0.0,0.0,0.017008,0.162075,0.0,0.003175,0.003775,0.021995,0.0,0.306521,0.0,0.001958,-0.025446,-1.970196,1.788404,0.0,0.02195,0.163172,-0.010143,11.624220,0.677252,-6.899207,7.279842,0.111101


In [ ]:
from sklearn.metrics import r2_score,mean_squared_error, mean_absolute_error

for i in range(final_df_norm.shape[0]):
  for j in range(final_df_norm.shape[0]):
    if i != j:
      r2 = r2_score(final_df_norm.iloc[i][:].to_numpy(),final_df_norm.iloc[j][:].to_numpy())
      mse = mean_squared_error(final_df_norm.iloc[i][:].to_numpy(),final_df_norm.iloc[j][:].to_numpy())
      mae = mean_absolute_error(final_df_norm.iloc[i][:].to_numpy(),final_df_norm.iloc[j][:].to_numpy())

      name1 = "XGBoost"
      name2 = "Random Forest"

      print("r2 Score for {} and {} : {}".format(name1,name2,r2))
      print("Mean squared error for {} and {} : {}".format(name1,name2,mse))
      print("Mean absolute error for {} and {} : {}".format(name1,name2,mae))
      print("\n")

r2 Score for XGBoost and Random Forest : -0.7113408837755708
Mean squared error for XGBoost and Random Forest : 20.297307542526536
Mean absolute error for XGBoost and Random Forest : 0.9431450374092838


r2 Score for XGBoost and Random Forest : -0.7161587159293241
Mean squared error for XGBoost and Random Forest : 20.297307542526536
Mean absolute error for XGBoost and Random Forest : 0.9431450374092838


